In [74]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#from joblib import dump, loada
import joblib
import csv

In [75]:
# Carga datos y crea variables dummy
df = pd.read_csv('juadores_fantasy.csv.csv')
# Guarda nombre original de columna
pos_col = 'posicion'
#categorical_cols = ['posicion', 'equipo']  
#df = pd.get_dummies(df, columns=categorical_cols)

In [76]:
# Codifica posicion y equipo a variables dummy
df = pd.get_dummies(df, columns=['posicion', 'equipo'])

In [77]:
# Obtén los nombres de las columnas dummy creadas
pos_dummies = [c for c in df.columns if c.startswith('posicion_')]

In [78]:
#Solicita entrada de posición
posicion = "delantero"


In [79]:
# Filtra usando una de las columnas dummy
df_filtrado = df[df[pos_dummies[0]] == 1]

In [80]:
# Separa las features de las variables a predecir  
X = df.drop(['Nombre', 'Total_puntos', 'Precio'], axis=1)
y_puntos = df['Total_puntos'] 
y_precio = df['Precio']

In [81]:
# Divide los datos en entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.3, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.3, random_state=42)

In [82]:
# Entrena un modelo para predecir puntos
rf_puntos = RandomForestRegressor()
rf_puntos.fit(X_train, y_puntos_train)

RandomForestRegressor()

In [83]:
# Entrena un modelo para predecir precio 
rf_precio = RandomForestRegressor(n_estimators=100)
rf_precio.fit(X_train, y_precio_train)

RandomForestRegressor()

In [84]:
# Realiza predicciones en los datos de test
y_puntos_pred = rf_puntos.predict(X_test)
y_precio_pred = rf_precio.predict(X_test)

In [85]:
# Evalúa los modelos
print("Puntos R2 Score:", rf_puntos.score(X_test, y_puntos_test))
mse = mean_squared_error(y_puntos_test, y_puntos_pred) 
print("Puntos MSE:", mse)

print("Precio R2 Score:", rf_precio.score(X_test, y_precio_test))
mse = mean_squared_error(y_precio_test, y_precio_pred)
print("Precio MSE:", mse)

Puntos R2 Score: 0.7158795292831184
Puntos MSE: 349.9556008057421
Precio R2 Score: 0.6726921194127673
Precio MSE: 11873190026916.98


In [86]:
# Obtén las predicciones ordenadas para encontrar los mayores
indices_puntos = y_puntos_pred.argsort()[::-1]
indices_precio = y_precio_pred.argsort()[::-1]

In [87]:
print(f"\n2 jugadores {posicion} con mayor puntaje predicho:")

for i in indices_puntos[:2]:

  nombre = df.loc[i,'Nombre']
  puntos = y_puntos_pred[i]
  precio = y_precio_pred[i]

  print(nombre, puntos, precio)


2 jugadores delantero con mayor puntaje predicho:
R. Alcaraz 139.85 19517210.0
R. Lewandowski 136.81 17585270.0


In [88]:
print(f"\n2 jugadores {posicion} con mayor precio predicho:")
for i in indices_precio[:2]:
  print(df.loc[i,'Nombre'], y_precio_pred[i])


2 jugadores delantero con mayor precio predicho:
R. Alcaraz 19517210.0
R. Lewandowski 17585270.0


In [89]:
with open('sugerencia.csv', 'w', newline='') as file:
  writer = csv.writer(file)

  writer.writerow(['Nombre', 'Puntos', 'Precio']) 
  
  for i in indices_puntos[:2]:
    nombre = df.loc[i,'Nombre']
    puntos = y_puntos_pred[i] 
    precio = y_precio_pred[i]
    
    writer.writerow([nombre, puntos, precio])

In [90]:
joblib.dump(rf_puntos, 'random_forest.pkl')

['random_forest.pkl']